In [1]:
from pliers.stimuli import VideoStim, AudioStim
from pliers.converters import IBMSpeechAPIConverter, VideoToAudioConverter, WitTranscriptionConverter
from pliers.extractors import PredefinedDictionaryExtractor, merge_results, STFTAudioExtractor
from pliers.graph import Graph
from pliers import config
config.cache_transformers = False
import os
from moviepy.audio.io.AudioFileClip import AudioFileClip
import numpy as np
import matplotlib.pyplot as plt
from traceback import format_exc
from glob import glob
import pickle
%matplotlib inline

In [3]:
os.environ['IBM_USERNAME'] = "c7c08d05-f40b-433e-83a2-f9754094a4d6"
os.environ['IBM_PASSWORD'] = "Qw78vA2ebHkg"
video_path = "D:neuroscout/datasets/hcp/movie_stimulus/Post_20140821_version/*.mp4"

In [8]:
# Extract audio track from each video, and split it in three segments
conv = VideoToAudioConverter()
videos = glob(video_path)
for v in videos:
    video = VideoStim(v)
    name = os.path.basename(v).split('.')[0]
    audio = conv.transform(video)
    for i, sc in enumerate([(0, 300), (300, 600), (600, int(audio.duration))]):
        subclip = audio.clip.subclip(*sc)
        subclip.write_audiofile('scratch/%s_%d.wav' % (name, i+1))

[MoviePy] Writing audio in D:neuroscout/datasets/hcp/movie_stimulus/Post_20140821_version\7T_MOVIE1_CC1_v2.wav


100%|██████████████████████████████████████████████████████████████████████████| 20309/20309 [00:16<00:00, 1259.44it/s]


[MoviePy] Done.
[MoviePy] Writing audio in scratch/7T_MOVIE1_CC1_v2_1.wav


100%|████████████████████████████████████████████████████████████████████████████| 6615/6615 [00:04<00:00, 1390.28it/s]


[MoviePy] Done.
[MoviePy] Writing audio in scratch/7T_MOVIE1_CC1_v2_2.wav


...

[MoviePy] Done.
[MoviePy] Writing audio in scratch/7T_MOVIE4_HO2_v2_2.wav


100%|████████████████████████████████████████████████████████████████████████████| 6615/6615 [00:04<00:00, 1415.25it/s]


[MoviePy] Done.
[MoviePy] Writing audio in scratch/7T_MOVIE4_HO2_v2_3.wav


100%|████████████████████████████████████████████████████████████████████████████| 6638/6638 [00:04<00:00, 1414.73it/s]


[MoviePy] Done.


In [4]:
# Convert to ComplexTextStim and update durations
texts = []
conv = IBMSpeechAPIConverter()
for i, f in enumerate(glob('scratch/*.wav')):
    seg_num = int(f[-5])-1
    print(f, "segment %d" % seg_num)
    audio = AudioStim(f)
    print("Transforming...")
    try:
        cts = conv.transform(audio)
        inc = seg_num*300
        for t in cts:
            t.onset += inc
        texts.append(cts)
    except:
        print(format_exc())


scratch\7T_MOVIE1_CC1_v2_1.wav segment 0
Transforming...
Traceback (most recent call last):
  File "<ipython-input-4-23104510e5b2>", line 10, in <module>
    cts = conv.transform(audio)
  File "C:\Users\aid338\AppData\Roaming\Python\Python35\site-packages\pliers\transformers.py", line 49, in wrapper
    result = transform(self, stim, *args, **kwargs)
  File "C:\Users\aid338\AppData\Roaming\Python\Python35\site-packages\pliers\transformers.py", line 87, in transform
    result = self._transform(validated_stim, *args, **kwargs)
  File "C:\Users\aid338\AppData\Roaming\Python\Python35\site-packages\pliers\converters\base.py", line 25, in _transform
    return self._convert(stim, *args, **kwargs)
  File "C:\Users\aid338\AppData\Roaming\Python\Python35\site-packages\pliers\converters\api.py", line 95, in _convert
    timestamps = result['results'][0]['alternatives'][0]['timestamps']
KeyError: 'results'

scratch\7T_MOVIE1_CC1_v2_2.wav segment 1
Transforming...
scratch\7T_MOVIE1_CC1_v2_3.wav s

In [5]:
i = 0
f = 'scratch\7T_MOVIE1_CC1_v2_1.wav'
seg_num = int(f[-5])-1
print(f, "segment %d" % seg_num)
audio = AudioStim(f)
print("Transforming...")
try:
    cts = conv.transform(audio)
    inc = seg_num*300
    for t in cts:
        t.onset += inc
    texts.append(cts)
except:
    print(format_exc())

scratchT_MOVIE1_CC1_v2_1.wav segment 0


OSError: MoviePy error: failed to read the duration of file scratchT_MOVIE1_CC1_v2_1.wav.
Here are the file infos returned by ffmpeg:

ffmpeg version N-67100-g6dc99fd Copyright (c) 2000-2014 the FFmpeg developers
  built on Oct 22 2014 22:01:57 with gcc 4.9.1 (GCC)
  configuration: --enable-gpl --enable-version3 --disable-w32threads --enable-avisynth --enable-bzlib --enable-fontconfig --enable-frei0r --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libfreetype --enable-libgme --enable-libgsm --enable-libilbc --enable-libmodplug --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-librtmp --enable-libschroedinger --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvo-aacenc --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --e

In [6]:
%debug

> c:\users\aid338\appdata\roaming\python\python35\site-packages\moviepy-0.2.2.11-py3.5.egg\moviepy\video\io\ffmpeg_reader.py(270)ffmpeg_parse_infos()
    268             raise IOError(("MoviePy error: failed to read the duration of file %s.\n"
    269                            "Here are the file infos returned by ffmpeg:\n\n%s")%(
--> 270                               filename, infos))
    271 
    272     # get the output line that speaks about video

ipdb> exit


In [13]:
# Pickle the ComplexTextStims for later use
for i, f in enumerate(glob('scratch/*.wav')):
    name = 'scratch/' + os.path.basename(f).split('.')[0] + '.pkl'
    print("Saving to %s." % name)
    pickle.dump(texts[i], open(name, 'wb'))

Saving to scratch/7T_MOVIE1_CC1_v2_1.pkl.
Saving to scratch/7T_MOVIE1_CC1_v2_2.pkl.
Saving to scratch/7T_MOVIE1_CC1_v2_3.pkl.
Saving to scratch/7T_MOVIE2_HO1_v2_1.pkl.
Saving to scratch/7T_MOVIE2_HO1_v2_2.pkl.
Saving to scratch/7T_MOVIE2_HO1_v2_3.pkl.
Saving to scratch/7T_MOVIE3_CC2_v2_1.pkl.
Saving to scratch/7T_MOVIE3_CC2_v2_2.pkl.
Saving to scratch/7T_MOVIE3_CC2_v2_3.pkl.
Saving to scratch/7T_MOVIE4_HO2_v2_1.pkl.
Saving to scratch/7T_MOVIE4_HO2_v2_2.pkl.


IndexError: list index out of range

In [19]:
# Load from pickles
ctstims = []
for i, f in enumerate(glob('scratch/*.pkl')):
    ctstims.append(pickle.load(open(f, 'rb')))

## Feature extraction

In [ ]:
# Extract text features
dict_ext = PredefinedDictionaryExtractor(['aoa/AoA_Kup_lem', 'SUBTLEXusfrequencyabove1/Lg10WF',
                                          'concreteness/Conc.M', 'affect/V.Mean.Sum'])

In [20]:

dfs = []
for start, end in [(0, 3), (3, 6), (6, 9), (9, 12)]:
    text_features = dict_ext.transform(ctstims[start:end])
    from itertools import chain
    df = merge_results(chain(*text_features))
    dfs.append(df)

Stim: 0it [00:00, ?it/s]
Stim: 0it [00:00, ?it/s]
Stim: 10it [00:00, 384.15it/s]
Stim: 0it [00:00, ?it/s]
Stim: 4it [00:00, 1976.35it/s]
Stim: 0it [00:00, ?it/s]
Stim: 3it [00:00, 60.00it/s]
Stim: 0it [00:00, ?it/s]
Stim: 0it [00:00, ?it/s]
Stim: 9it [00:00, 1790.57it/s]
Stim: 0it [00:00, ?it/s]
Stim: 9it [00:00, 1500.29it/s]
Stim: 0it [00:00, ?it/s]
Stim: 3it [00:00, 115.51it/s]
Stim: 0it [00:00, ?it/s]
Stim: 0it [00:00, ?it/s]
Stim: 1it [00:00, 999.60it/s]
Stim: 0it [00:00, ?it/s]
Stim: 2it [00:00, 2001.58it/s]
Stim: 0it [00:00, ?it/s]
Stim: 3it [00:00, 200.36it/s]
Stim: 0it [00:00, ?it/s]
Stim: 0it [00:00, ?it/s]
Stim: 1it [00:00, 111.11it/s]


In [4]:
for i, d in enumerate(dfs):
    d.to_csv('text_features_video_%d.csv' % (i+1), index=False)

### Equalizer results

In [10]:
# Equalizer
freq_bins = [(16, 60), (60, 250), (250, 500), (500, 1000), (1000, 2000), (2000, 5000)]
stft_ext = STFTAudioExtractor(frame_size=0.2, hop_size=0.05, freq_bins=freq_bins)
for i, v in enumerate(glob(video_path)):
    video = VideoStim(v)
    df = stft_ext.transform(video).to_df()
    df.to_csv('freq_spect_clip{}.csv'.format(i+1), index=False)

[MoviePy] Writing audio in D:neuroscout/datasets/hcp/movie_stimulus/Post_20140821_version\7T_MOVIE1_CC1_v2.wav


100%|██████████████████████████████████████████████████████████████████████████| 20309/20309 [00:16<00:00, 1248.85it/s]


[MoviePy] Done.
[MoviePy] Writing audio in D:neuroscout/datasets/hcp/movie_stimulus/Post_20140821_version\7T_MOVIE2_HO1_v2.wav


100%|██████████████████████████████████████████████████████████████████████████| 20242/20242 [00:15<00:00, 1287.81it/s]


[MoviePy] Done.
[MoviePy] Writing audio in D:neuroscout/datasets/hcp/movie_stimulus/Post_20140821_version\7T_MOVIE3_CC2_v2.wav


100%|██████████████████████████████████████████████████████████████████████████| 20176/20176 [00:15<00:00, 1288.45it/s]


[MoviePy] Done.
[MoviePy] Writing audio in D:neuroscout/datasets/hcp/movie_stimulus/Post_20140821_version\7T_MOVIE4_HO2_v2.wav


100%|██████████████████████████████████████████████████████████████████████████| 19868/19868 [00:15<00:00, 1299.12it/s]


[MoviePy] Done.
